### Step 0: Extract the dataset

In [1]:
import zipfile
import os

# Set path to your zip file and extract location
zip_path = '/content/drive/MyDrive/archive.zip'
extract_dir = '/content/drive/MyDrive/chicken_duck_dataset/'

# Extract zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Zip file extracted to:", extract_dir)

Zip file extracted to: /content/drive/MyDrive/chicken_duck_dataset/


### Step 1: Install and import dependencies

In [2]:
!pip install torch torchvision matplotlib scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [3]:
import os
import glob
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.folder import default_loader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report

### Step 2: Mount Google Drive (optional, but useful)


In [4]:
# Step 2: Mount Google Drive (optional, but useful)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Step 3: Organize your image dataset like this in Drive:

In [5]:
# Class names
class_names = ['chicken', 'duck']

# Path to dataset
base_path = '/content/drive/MyDrive/chicken_duck_dataset'

# Image transforms (ResNet expects 224x224, normalized)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

### Step 4: Load dataset and create dataloaders

In [6]:
class DuckChickenDataset(Dataset):
    def __init__(self, chicken_dir, duck_dir, transform=None):
        self.chicken_images = glob.glob(os.path.join(chicken_dir, '*'))
        self.duck_images = glob.glob(os.path.join(duck_dir, '*'))
        self.images = self.chicken_images + self.duck_images
        self.labels = [0] * len(self.chicken_images) + [1] * len(self.duck_images)
        self.transform = transform
        self.loader = default_loader

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        path = self.images[idx]
        label = self.labels[idx]
        image = self.loader(path)
        if self.transform:
            image = self.transform(image)
        return image, label

In [7]:
# Load datasets
train_dataset = DuckChickenDataset(
    chicken_dir=os.path.join(base_path, 'chicken/train'),
    duck_dir=os.path.join(base_path, 'duck/train'),
    transform=transform
)

val_dataset = DuckChickenDataset(
    chicken_dir=os.path.join(base_path, 'chicken/val'),
    duck_dir=os.path.join(base_path, 'duck/val'),
    transform=transform
)

test_dataset = DuckChickenDataset(
    chicken_dir=os.path.join(base_path, 'chicken/test'),
    duck_dir=os.path.join(base_path, 'duck/test'),
    transform=transform
)

In [8]:
# Loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(f"Loaded data: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")

Loaded data: Train=897, Val=161, Test=482


### Step 5: Load pre-trained ResNet18 and modify final layer

In [9]:
import warnings
warnings.filterwarnings("ignore")

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load pretrained ResNet18 and modify last layer
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # freeze everything

# Replace final layer with 2-class output
model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)

Using device: cuda


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 142MB/s]


### Step 6: Define loss, optimizer, and train the model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

def evaluate_model(model, loader, phase='Test'):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    print(f"\n📊 {phase} Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"📈 Epoch [{epoch+1}/{epochs}] - Loss: {running_loss/len(train_loader):.4f}")
        evaluate_model(model, val_loader, phase='Validation')

In [ ]:
# Train for 5 epochs
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)

# Final evaluation on test data
evaluate_model(model, test_loader, phase='Test')

📈 Epoch [1/10] - Loss: 0.1482

📊 Validation Classification Report:
              precision    recall  f1-score   support

     chicken       0.88      0.88      0.88        52
        duck       0.94      0.94      0.94       109

    accuracy                           0.93       161
   macro avg       0.91      0.91      0.91       161
weighted avg       0.93      0.93      0.93       161

📈 Epoch [2/10] - Loss: 0.1897

📊 Validation Classification Report:
              precision    recall  f1-score   support

     chicken       0.98      0.79      0.87        52
        duck       0.91      0.99      0.95       109

    accuracy                           0.93       161
   macro avg       0.94      0.89      0.91       161
weighted avg       0.93      0.93      0.92       161

📈 Epoch [3/10] - Loss: 0.1159

📊 Validation Classification Report:
              precision    recall  f1-score   support

     chicken       0.98      0.77      0.86        52
        duck       0.90      0.99   